In [2]:
import numpy as np

In [3]:
#Simple 2D quadratic

def f1(x): #Function
    return ((x[0]-0.25)**2)+((x[1]-0.25)**2)

def gradf1(x): #Gradient
    g = [2*(x[0]-0.25),2*(x[1]-0.25)]
    return np.asarray(g)

In [4]:
#General Initializations for simple Quadratic
w_init_q = np.array([0,1]) #Initial point
L1 = 2 #Lipschitz for quadratic
S1 = np.array([[-1,0],[1,0],[0,1]]) #Constrained Set Vertices
alpha1 = np.zeros(S1.shape[0]) #Weight Vector for AFW
quad_epsilon_tol = 0.05 #tolerance to drop vertices
max_iter = 500 #Max number of iterations

In [7]:
#Away Steps for Simple Quadratic

def LP1min(x,S):
    idx = np.argmin(np.matmul(S,gradf1(x)))
    return S[idx],idx

def LP1max(x,S):
    S_matrix = []
    S_indices = []
    for key,value in S.items():
        S_matrix.append(value)
        S_indices.append(key)
    S_matrix = np.array(S_matrix)
    idx = np.argmax(np.matmul(S_matrix,gradf1(x)))   
    return S_matrix[idx],S_indices[idx]

def FW_awaysteps(w_init,S,alpha_0,L,max_iter,epsilon2):
    w_0,index = LP1min(w_init,S)
    alpha_0[index] = 1
    S_0 = {} #Active Set
    S_0[index] = w_0 #Adding vertex to active set
    it = 0
    #flag = None
    itr = []
    coordinates = []
    f_values = []
    #endFlag = False
    count1 = 0 #Count no of drop steps
    count = 0 #Count no of away step selection
    while (it <= max_iter): 
        vt_FW,i_FW = LP1min(w_0,S) 
        vt_A,i_A = LP1max(w_0,S_0)
        if np.dot(gradf1(w_0),w_0-vt_FW)>=np.dot(gradf1(w_0),vt_A-w_0):
            d_t = w_0 - vt_FW
            gamma_max = 1
            #flag = True
        else:
            d_t = vt_A - w_0
            gamma_max = alpha_0[i_A]/(1-alpha_0[i_A])
            count +=1 
            #flag = False
        gamma_t = min(np.dot(gradf1(w_0),d_t)/(L*(np.linalg.norm(d_t)**2)),gamma_max)
        w_0 = w_0 - gamma_t*d_t
        if np.dot(gradf1(w_0),w_0-vt_FW)>=np.dot(gradf1(w_0),vt_A-w_0):
            alpha_0 = (1-gamma_t)*alpha_0
            if i_FW in S_0:
                alpha_0[i_FW] = alpha_0[i_FW]+gamma_t
            else:
                alpha_0[i_FW] = gamma_t
            if gamma_t <1:
                S_0[i_FW] = vt_FW
            else:
                S_0.clear()
                S_0[i_FW] = vt_FW
        else:
            alpha_0 = (1+gamma_t)*alpha_0
            if i_A in S_0:
                alpha_0[i_A] = alpha_0[i_A]-gamma_t
            if abs(gamma_t-gamma_max)<10*epsilon2: #tol to ensure weight zeroing
                #print("yes",gamma_t,gamma_max)
                alpha_0[i_A] = 0
                S_0.pop(i_A)
                count1+=1 
        if it%100 == 0:
            print("Coordinates,gamma_t",w_0,gamma_t)
        itr.append(it)
        coordinates.append(w_0)
        f_values.append(f1(w_0))
        it = it+1
    return coordinates,itr,f_values,count1

In [8]:
coordinates1,itr1,f_values1,count1 = FW_awaysteps(w_init_q,S1,alpha1,L1,max_iter,quad_epsilon_tol)

Coordinates,gamma_t [0.25 0.  ] 0.375
Coordinates,gamma_t [0.25 0.25] 0.0
Coordinates,gamma_t [0.25 0.25] 0.0
Coordinates,gamma_t [0.25 0.25] 0.0
Coordinates,gamma_t [0.25 0.25] 0.0
Coordinates,gamma_t [0.25 0.25] 0.0


In [9]:
#No of drop steps
print("Drop Steps Count:",count1)

Drop Steps Count 1
